# Day 5

In [1]:
import os

In [43]:
input_file_path = os.path.join(".", "day05.txt")
with open(input_file_path, 'r') as reader:
    input_data = reader.read()

## Part 1
Count the number of points where two or more lines overlap

### The naive way
Using a bit array

In [44]:
import numpy as np
import pandas as pd

instructions = []
width = height = 0

# First pass: parse all input lines and determine field dimensions
for instruction in input_data.split("\n"):
    if len(instruction) < 1:
        continue

    origin, target = instruction.split(" -> ")
    coordinates = list(map(int, origin.split(",") + target.split(",")))
    instructions.append(coordinates)

    width = max(width, coordinates[0], coordinates[2])
    height = max(height, coordinates[1], coordinates[3])

width += 1
height += 1

field = np.zeros((width, height))
print(f"Field dimension is {width:,} x {height:,}, using {field.nbytes / 1024:0,.1f}KB of memory")


Field dimension is 990 x 991, using 7,664.8KB of memory


In [46]:
# Second pass: mark lines
for (x1, y1, x2, y2) in instructions:
    # Horizontal line
    if x1 == x2:
        # print(f"H: {x1}, {y1} -> {x2}, {y2}")
        if y1 > y2:
            y1, y2 = y2, y1
        field[y1:y2+1, x1] += 1

    # Vertical line
    elif y1 == y2:
        # print(f"V: {x1}, {y1} -> {x2}, {y2}")
        if x1 > x2:
            x1, x2 = x2, x1
        field[y1, x1:x2+1] += 1

    else:
        # print(f"X: {x1}, {y1} -> {x2}, {y2}")
        continue
    
# display(pd.DataFrame(field).astype(int).replace(0, "."))
print(f"More than two overlaps: {np.sum(field > 1):,}")


More than two overlaps: 100,746


## Part 2
Find last winning bingo card

In [155]:
import numpy as np

# Read in raw data and squash redundant whitespace
raw_data = (
    input_data
        .replace("  ", " ")
        .replace("\n ", "\n")
        .split("\n")
)

# First line holds the bingo number draws
arr_draws = np.array(raw_data[0].split(",")).astype(int)

# Subsequent lines hold the bingo cards
n = len(raw_data) - 1
arr_cards = np.array([
    arr.split(" ")
    for i in range(2, n, 6)
    for arr in raw_data[i:i + 5]
]).astype(int).reshape((n // 6, 5, 5))


In [156]:
# Apply draws
idx_drawn = np.full_like(arr_cards, fill_value=False, dtype=bool)
arr_winning_cards = np.full(arr_cards.shape[0], fill_value=False, dtype=bool)
for draw in arr_draws:
    idx_drawn |= (arr_cards == draw)

    idx_bingo_row = idx_drawn.all(axis=2).any(axis=-1)
    idx_bingo_col = idx_drawn.all(axis=1).any(axis=-1)
    arr_pre = arr_winning_cards.copy()

    # Winning bingo row or column
    if idx_bingo_row.any():
        arr_winning_cards |= idx_bingo_row
    if idx_bingo_col.any():
        arr_winning_cards |= idx_bingo_col

    # Last winning card is found
    if np.sum(arr_winning_cards) == arr_cards.shape[0]:
        # Last winning board
        a = np.ma.masked_where(idx_drawn, arr_cards)[~arr_pre].sum()
        print(f"{a} x {draw} = {a * draw}")
        break


365 x 87 = 31755
